# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력받는다

=> A:B = C:answer 관계에 대응하는 answer를 찾는 게임을 만든다<br>
ex) A: 산, B: 바다, C: 나무 / D -> 물

In [1]:
import random
from gensim.models import KeyedVectors

# Word2Vec 모델 로드
# model = KeyedVectors.load_word2vec_format('GoogleNews_vecs.bin.gz', binary=True)
model = KeyedVectors.load_word2vec_format('naver_movie_ratings_w2v')

In [2]:
def find_similar_pair():
    vocab = list(model.key_to_index.keys())
    while True:
        a = random.choice(vocab)
        for b in vocab:
            if model.similarity(a, b) >= 0.8 and a != b:
                return a, b

In [ ]:
def find_corresponding_pair(a, b, topn=1):
    try:
        # a:b 관계와 유사한 관계를 가지는 단어 쌍 찾기
        results = model.most_similar(positive=[b, a], negative=[a], topn=topn*10)
        
        valid_pairs = []
        for c, _ in results:
            try:
                # c:d 관계가 a:b 관계와 유사한지 확인
                d, score = model.most_similar(positive=[b, c], negative=[a], topn=1)[0]
                if score > 0.5 and c != d:  # 유사도 임계값 및 같은 단어 제외
                    valid_pairs.append((c, d, score))
            except:
                continue 
            
            if len(valid_pairs) >= topn:
                break
        
        if valid_pairs:
            # score를 기준으로 내림차순 정렬하고 가장 높은 score의 c, d 반환
            best_pair = max(valid_pairs, key=lambda x: x[2])
            return best_pair[0], best_pair[1]  # c, d만 반환
        else:
            return None, None  # 유효한 쌍이 없을 경우
    except:
        return None, None

In [4]:
def play_analogy_game(a, b, c):
    
    print(f"관계 [ {a} : {b} = {c} : ? ]")
    user_answer = input("위 관계를 만족하는 단어를 입력하세요: ")
    
    expected = model.most_similar(positive=[b, c], negative=[a], topn=1)[0][0]
    similarity = model.similarity(user_answer, expected)
    
    print(f"모델이 예측한 가장 적합한 단어: {expected}")
    print(f"당신의 답변과 모델 예측의 유사도: {similarity:.2f}")
    
    if similarity > 0.5:
        print("괜찮은 답변이네요 :)")
    else:
        print("아쉽네요. 더 생각해보세요.")


In [5]:
# 게임 실행
a, b = find_similar_pair()
c, d = find_corresponding_pair(a, b)

play_analogy_game(a, b, c)

관계 [ 예견 : 조롱 = 행세 : ? ]
모델이 예측한 가장 적합한 단어: 목회자
당신의 답변과 모델 예측의 유사도: 0.63
괜찮은 답변이네요 :)
